In [1]:
from __future__ import print_function,division
from future.utils import iteritems
from builtins import range
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [2]:
train = pd.read_csv('C:\\Users\\ac36345\\Desktop\\nlp code\\r8-train-all-terms.txt',header=None,sep='\t')
test = pd.read_csv('C:\\Users\\ac36345\\Desktop\\nlp code\\r8-test-all-terms.txt',header=None,sep='\t')
train.columns = ['label','contents']
test.columns = ['label','contents']

In [3]:
class GloveVectorizer:
    def __init__(self):
        print('loadind word vectors...')
        word2vec = {}
        embedding = []
        idx2word = []
        with open('C:\\Users\\ac36345\\Desktop\\nlp code\\glove.6B\\glove.6B.50d.txt',encoding="utf8") as f:
            for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:],dtype='float32')
                word2vec[word] = vec
                embedding.append(vec)
                idx2word.append(word)
        print('found %s word vectors.' %len(word2vec))
        self.word2vec= word2vec
        self.embedding = np.array(embedding)
        self.word2idx = {v:k for k,v in enumerate(idx2word)}
        self.V, self.D = self.embedding.shape
    
    def fit(self,data):
        pass
                        
    def transform(self,data):
        X = np.zeros((len(data),self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            for word in tokens:
                if word in self.word2vec:
                    vec = self.word2vec
                    vecs.append(vec)
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount +=1
            n += 1
        print('number of samples with no words found: %s / %s ' %(emptycount,len(data)))
        return X
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)
    
                        
        


In [4]:
#word2vec
class word2VecVectorizer:
    def __init__(self):
        print('loadind word vectors...')
        self.word_vectors = KeyedVectors.load_word2vec_format('C:\\Users\\ac36345\\Desktop\\GoogleNews-vectors-negative300.bin.gz',binary=True)
        print("finished loading in word vectors")
    
    def fit(self,data):
        pass
                        
    def transform(self,data):
        v = self.word_vectors.get_vector('king')
        self.D = v.shape[0]
        X = np.zeros((len(data),self.D))
        n = 0
        emptycount = 0
        for sentence in data:
            tokens = sentence.lower().split()
            vecs = []
            m=0
            for word in tokens:
                try:
                    vec = self.word_vectors.get_vector(word)
                    vecs.append(vec)
                    m +=1
                except KeyError:
                    pass
            if len(vecs) > 0:
                vecs = np.array(vecs)
                X[n] = vecs.mean(axis=0)
            else:
                emptycount +=1
            n += 1
        print('number of samples with no words found: %s / %s ' %(emptycount,len(data)))
        return X
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)
    

In [6]:
vectorizer = word2VecVectorizer()
Xtrain = vectorizer.fit_transform(train.contents)
Ytrain = train.label

Xtest = vectorizer.transform(test.contents)
Ytest = test.label



loadind word vectors...


C:\Users\ac36345\AppData\Local\Continuum\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


finished loading in word vectors
number of samples with no words found: 0 / 5485 
number of samples with no words found: 0 / 1152 


In [7]:
model = RandomForestClassifier(n_estimators=200)
model.fit(Xtrain,Ytrain)
print('train score:',model.score(Xtrain,Ytrain))
print('test score:',model.score(Xtest,Ytest))

train score: 0.9992707383773929
test score: 0.9375
